In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import math
from datetime import datetime

#### Import Kestrel data and change year to 2015

In [9]:
dtypes=[datetime, float]
kes15 = pd.read_excel('Kestrel_data2015.xlsx',names=['time','wind_speed'],header=3)
kes15['time']=kes15['time'].apply((lambda x: x.replace(year=2015)))
kes15.head()

,time,wind_speed
0,2015-03-11 12:15:00,0.0
1,2015-03-11 12:16:00,0.0
2,2015-03-11 12:17:00,0.0
3,2015-03-11 12:18:00,0.2
4,2015-03-11 12:19:00,0.3


#### Import 2015 Sonic data 

In [ ]:
son15 = pd.read_csv('2015_sonic_1230_start.txt', sep=",", header=None)
data.columns = ['u','v','w','sound_speed','temp']
kes15 = pd.read_excel('Kestrel_data2015.xlsx',names=['u','v','w','sound_speed','temp'],header=3)